# Using OLS and Multiple Regression to Predict Housing Prices
## Alex Lee

I will investigate the relationship between housing prices and the physical characteristics of a home. I used the hprice.xls data set which was collected from the real estate pages of the Boston Globe in 1990 (these are homes selling in the Boston, MA area). It contains data on the selling price (price) of the house (in$1000), the size (sqrft) of the house in square feet, the number of bedrooms (bdrms), the size of the lot (lotsize) in square feet, and a dummy variable (colonial) which  is equal to 1 if the home was colonial style. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer

In [2]:
data = pd.read_excel('hprice.xls')

In [3]:
data.head()

,price,bdrms,lotsize,sqrft,colonial
0,300.0,4,6126,2438,1
1,370.0,3,9903,2076,1
2,191.0,3,5200,1374,0
3,195.0,3,4600,1448,1
4,373.0,4,6095,2514,1


In [4]:
data.shape

(101, 5)

## Simple Regression Model
Running a regression of selling price (price) on the size of the home (sqrft) and the number of bedrooms (bdrms).

In [7]:
price = data['price']
x = data.drop(columns = 'price')

Using the SciKit learn package for linear regression.

In [8]:
from sklearn.linear_model import LinearRegression
#Selecting only the sqrft column
model = LinearRegression().fit(x[['sqrft']], price)

Slope of the regression line:

In [14]:
model.coef_[0]

0.13930515867812018

In [10]:
model.intercept_

12.403224388309297

Prediction for a 2000 sqrft home. Can be run with any value of sqrft. 

In [13]:
1000 * model.predict([[2000]])[0]

291013.5417445497

Now, I will look at the estimated increase in price for a house with one more bedroom, holding square footage constant. This will help to see the effect of adding another regressor to the equation. 

In [15]:
features = x[['bdrms', 'sqrft']]
model = LinearRegression().fit(features, price)

In [16]:
model.coef_

array([14.41436051,  0.12808873])

The expected increase in price for a house with an additional bedroom holding sqrft constant is:

In [20]:
print('$ ' + str(model.coef_[0] * 1000))

$ 14414.360510822653


Percentage of the variation in price explained by square footage and the number of bedrooms:

In [22]:
print(data.var())

price       9.559607e+03
bdrms       6.704950e-01
lotsize     9.138198e+07
sqrft       3.043069e+05
colonial    2.108911e-01
dtype: float64


In [25]:
var = data.var()
total = sum(var)
sb_var = var[1] + var[3]
print("Percent Variation " + str((sb_var/total) * 100) + "%")

Percent Variation 0.3318662132670174%


As seen, the percent variation is only .33%

### Defining an OLS Regression Model

In [28]:
#Example square feet and bedrooms
sqrft = 2438
bdrms = 4

def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    
    Parameters
    -----------
    thetas: a 1D vector representing the parameters of our model ([theta1, theta2, ...])
    X: a 2D dataframe of numeric features (may also be a 2D numpy array)
    
    Returns
    -----------
    A 1D vector representing the linear combination of thetas and features as defined above.
    """
    return np.dot(X, thetas)

In [29]:
from scipy.optimize import minimize
##OLS
def l2(y, y_hat):
    return (y - y_hat)**2
def minimize_average_loss(loss_function, model, X, y):
    """
    Minimize the average loss calculated from using different theta vectors, and 
    estimate the optimal theta for the model.
    
    Parameters
    -----------
    loss_function: either the squared or absolute loss functions defined above
    model: the model.
    X: a 2D dataframe (or numpy array) of numeric features (one-hot encoded)
    y: a 1D vector of tip amounts
    
    Returns
    -----------
    The estimate for the optimal theta vector that minimizes our loss
    """
    return minimize(lambda theta: np.mean(loss_function(y, model(theta,X))), x0= np.ones(X.shape[1]))['x']

In [30]:
coeffs = minimize_average_loss(l2, linear_model, features, price)

Predicted Price using OLS

In [33]:
housePrice = coeffs[0] * bdrms + coeffs[1] * sqrft
print ('$' + str(housePrice * 1000))

$351679.4195098176


The sqrft and bdrms column are correlated so there is a possibility of an ommitted variable bias (OVB) with sqrft and thus overpredicting price (having too much weight). Below is an analysis to see if this is the case. In our data set, the given house price with these parameters is $300k.

In [37]:
data['sqrft'].corr(data['bdrms'])

0.524224507321314

In [38]:
coeffs = minimize_average_loss(l2, linear_model, data[['sqrft']], price)
coeffs

array([0.14512089])

In [39]:
housePrice2 = coeffs[0] * sqrft
housePrice2

353.8047359589831

As shown, our model without bedrooms overpredicts the effect of square footage and has an OVB. 

### Testing for Multicolinearity
To do this, I will include lotsize in the original regression.

In [40]:
features2 = data[['bdrms', 'sqrft', 'lotsize']]
coeffs3 = minimize_average_loss(l2, linear_model, features2, price)
coeffs3

array([9.97008341e+00, 1.18851320e-01, 2.11072691e-03])

The lot size coefficient is shown above as the third item in the array (2.11e-03). This coefficient is very small showing that the effect of lot size on the regression does not have a huge effect. This could potentially indicate a problem with multicolinearity as the columns could be linear transformations of each other. Another test is shown below to analyze multicolinearity.

In [41]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data['feature'] = features2.columns
vif_data["VIF"] = [variance_inflation_factor(features2.values, i) for i in range(len(features2.columns))]
print(vif_data)

   feature        VIF
0    bdrms  17.294603
1    sqrft  17.728210
2  lotsize   1.910102


As seen above, bdrms and sqrft have very high levels of variance inflation factor, indicating that these variables are highly correlated. This is expected because as the number of bedrooms increases, the sqrft generally does too. These two features together likely lead to a model with high multicolinearity. 